# Importing articles from macrostructure.json

In [40]:
import json

with open("resources/macrostructure.json", "r") as macrostructure_file:
    articles_list = json.load(macrostructure_file)

# Creating the ontology

In [41]:
from owlready2 import owlready2, get_ontology, sync_reasoner_pellet, Thing, Not, And, default_world, AllDifferent, AllDisjoint, Imp

In [42]:
owlready2.JAVA_EXE='/usr/bin/java'

In [43]:
# Fetch root ontology
onto = get_ontology("http://authors_relations.org/onto.owl")

In [44]:
# Create ontology classes
with onto :
    class Author(Thing):
        pass

    class Article(Thing):
        is_a = [Not(onto.Author)]

    class is_author_of(Author >> Article): pass

    class is_written_by(Article >> Author): pass
    
    class cites(Author >> Author): pass

    class worked_with(Author >> Author): pass

    class is_cited_by(Author >> Author): pass

    class is_cited_in(Author >> Article): pass

    class CitedAuthor(Thing):
        equivalent_to = [And([Author, is_cited_by.some(Author), is_cited_in.some(Article)])]

In [46]:
from cleaner_utils.cleaner_custom import escape_iri, escape_value

# Populating individuals in ontology
with onto:
    for article in articles_list:
        uri = article['uri']
        urn = uri.replace("http://arxiv.org/pdf/cs/","")
        article_object = onto.Article(
            urn=escape_iri(urn),
            )
        for article_author in article['authors']:
            try:
                # Look for article_author (if already created in ontology)
                author_object = onto.search(iri="*{}".format(escape_iri(article_author)))[0]
            except IndexError:
                # Create it otherwise
                author_object = onto.Author(
                    name=escape_iri(article_author)
                )
            article_object.is_written_by.append(author_object)
        for reference in article['references']:
            for cited_author in reference['authors']:
                try:
                    # Look for cited_author (if already created in ontology)
                    cited_author_object = onto.search(iri="*{}".format(escape_iri(cited_author)))[0]
                except IndexError:
                    # Create it otherwise
                    cited_author_object = onto.Author(
                        name=escape_iri(cited_author)
                    )
                cited_author_object = onto.Author(
                    name=escape_iri(cited_author)
                )
                if article_object not in cited_author_object.is_cited_in:
                    cited_author_object.is_cited_in.append(article_object)
            
    print(onto.Article.instances())

[onto.article1, onto.article2, onto.article3, onto.article4, onto.article5, onto.article6, onto.article7, onto.article8, onto.article9, onto.article10, onto.article11, onto.article12, onto.article13, onto.article14, onto.article15, onto.article16, onto.article17, onto.article18, onto.article19, onto.article20, onto.article21, onto.article22, onto.article23, onto.article24, onto.article25, onto.article26, onto.article27, onto.article28, onto.article29, onto.article30, onto.article31, onto.article32, onto.article33, onto.article34, onto.article35, onto.article36, onto.article37, onto.article38, onto.article39, onto.article40, onto.article41, onto.article42, onto.article43, onto.article44, onto.article45, onto.article46, onto.article47, onto.article48, onto.article49, onto.article50, onto.article51, onto.article52, onto.article53, onto.article54, onto.article55, onto.article56, onto.article57, onto.article58, onto.article59, onto.article60, onto.article61, onto.article62, onto.article63, 

# Defining reasoner rules

In [50]:
# Creating the Imply rules
with onto:
    imp = Imp()
    # Defining a new rule : the author A of an article B from which another author C is cited, cites him
    imp.set_as_rule("""is_author_of(?a,?b), is_cited_in(?c, ?b) -> cites(?a, ?c) """)
    
    # Defining a new rule : If an author A cites an author B, the author B is cited by A
    imp.set_as_rule("""cites(?a, ?b) -> is_cited_by(?b, ?a) """)

    # Defining a new rule : If A and B are both authors of the same article, then A worked with B
    imp.set_as_rule("""is_author_of(?a,?c), is_author_of(?b,?c) -> worked_with(?a, ?b) """)

    # Defining a new rule (reciprocity) : If author A worked with author B, then B worked with A
    imp.set_as_rule("""worked_with(?a, ?b) -> worked_with(?b, ?a) """)

# Applying rules to infer new relations in the ontology

In [51]:
# Synchronizing reasoner to apply rules
with onto:
    sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /home/william/Programming/OntologyMaker/.venv/lib/python3.8/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/william/Programming/OntologyMaker/.venv/lib/python3.8/site-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/home/william/Programming/OntologyMaker/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/william/Programming/OntologyMaker/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/william/Programming/OntologyMaker/.venv/lib/python3.8/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/william/Programming/OntologyMaker/.venv/lib/python3.8/site-packages/owlready2/pellet/antlr-3.2.jar:/home/william/Programming/OntologyMaker/.venv/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/william/Programming/OntologyMaker/.venv/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/william/Programming/OntologyMaker/.v

# Saving the ontology in a file

In [12]:
# Save the ontology
onto.save(file = "authors_relations.owl")